In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dsaa-6100-titanic-dicision_tree/train.csv
/kaggle/input/dsaa-6100-titanic-dicision_tree/test.csv
/kaggle/input/dsaa-6100-titanic-dicision_tree/gender_submission.csv


# 1. **Data preprocessing**

In [ ]:
df = pd.read_csv('/kaggle/input/dsaa-6100-titanic-dicision_tree/train.csv')
tst_df = pd.read_csv('/kaggle/input/dsaa-6100-titanic-dicision_tree/test.csv')

In [60]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,B96 B98,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,B96 B98,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,B96 B98,0
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,24.0,1,2,W./C. 6607,23.4500,B96 B98,0
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,1


In [ ]:
df.isna().sum()

In [ ]:
modes = df.mode().iloc[0]
modes

In [ ]:
df.fillna(modes, inplace=True)

In [ ]:
import numpy as np
print(df.describe(include=(np.number)))
print(df.describe(include=[object]))
df

In [ ]:
df = df.replace({'Sex': {'female': 0, 'male': 1}})
df = df.replace({'Embarked': {'S': 0, 'C': 1, 'Q': 2}})

# 2. **Decision Tree**

In [ ]:
X = df.drop(columns=['Survived','Name','Ticket','Cabin'])
y = df['Survived']
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
model = DecisionTreeClassifier(max_depth=3, random_state=123)

In [75]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
a = pd.DataFrame()
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)
print('分类准确率为: %f%%' % (score * 100))


分类准确率为: 80.446927%


In [ ]:
features = X.columns
importances = model.feature_importances_

importances_df = pd.DataFrame()
importances_df['特征名称'] = features
importances_df['特征重要性'] = importances
importances_df.sort_values('特征重要性', ascending=False, inplace=True)
print(importances_df)

In [ ]:
tst_df

In [ ]:
tst_df.fillna(modes, inplace=True)
tst_df = tst_df.replace({'Sex': {'female': 0, 'male': 1}})
tst_df =tst_df.replace({'Embarked': {'S': 0, 'C': 1, 'Q': 2}})
tst_df_X = tst_df.drop(columns=['Name','Ticket','Cabin'])
y_pred = model.predict(tst_df_X)
tst_df['Survived']= y_pred

In [ ]:
sub_df = tst_df[['PassengerId','Survived']]
sub_df
sub_df.to_csv(f'submission.csv', index=False)

# 3. **Random forest**

In [85]:
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier(max_depth=5, n_estimators=7,min_samples_leaf=10, random_state=1)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)
score2 = accuracy_score(y_pred2, y_test)
print('分类准确率为: %f%%' % (score2 * 100))

分类准确率为: 77.653631%
